In [10]:
import numpy as np
import gurobipy as gp
from functions import *
from data_store import *
from Class_UAV import *
import time
from SDA_evaluations import *
from SAA import *
from SDA import *
from parameters import *
import os
from Benders import *
from SAA_original import *
from solving_subproblem import *

In [11]:
m = gp.Model("GB_ENV_Act")
m.setParam('OutputFlag', 0)

In [12]:
# This part, I want to do the out-of-sample test, to show the significance of considering the uncertainty of weather.

# The first step is to retrieve the optimal solution of not considering the weather uncertainty.
max_LP_list = [4 , 5 , 6]
max_DP_list = [1 , 2]
num_SmallDrone_list = [10 , 15 , 20]
num_LargeDrone_list = [1 , 2 , 3]

magnitude = [7.5, 6.8, 7.7, 7.6, 7.7, 6.8, 7.3, 7.0, 6.8, 7.7, 6.8, 7.7]
depth = [5.3, 14.4, 5.4, 16, 12.7, 8.7, 12.6, 8.2, 12.7, 12.7, 16.9, 8.7]

total_test = 3

directory = './output/'
if not os.path.exists(directory):
    os.makedirs(directory)

file_name = f'{directory}output_EVPI_test_{1}.out'

param = Parameters()
param.printlevel = 0

max_LP = param.p
max_DP = param.e
num_SmallDrone = param.K_SD
num_LargeDrone = param.K_LD

S = param.n_scenarios
pi = param.pi
T = param.T
e = param.e
p = param.p
Q_T = param.Q_T
c_SD = param.c_SD
c_LD = param.c_LD
v_T = param.v_T
tau = param.tau
K_SD = param.K_SD
K_LD = param.K_LD
num_candidates_depots = param.num_candidates_depots
num_candidates_launch_points = param.num_candidates_launch_points
num_gathering_points = param.num_gathering_points
printlevel = param.printlevel

print(f'Test {1} : max_LP = {max_LP} , max_DP = {max_DP} , num_SmallDrone = {num_SmallDrone} , num_LargeDrone = {num_LargeDrone}')
with open(file_name, 'a') as f:
    f.write(f'Test {1} : max_LP = {max_LP} , max_DP = {max_DP} , num_SmallDrone = {num_SmallDrone} , num_LargeDrone = {num_LargeDrone}\n')

# # Starting the test
# # Firstly, test with the formulation with the weather uncertainty
# SAA_objval , SAA_time , saa_uvw , g_SDLD = sample_average_approximation( 1 , param , earthquake_info=[magnitude,depth])
# print(f'SAA : {SAA_objval}')
# print(f'SAA time : {SAA_time}')
# # Secondly, test with the formulation without the weather uncertainty
# SAA_O_objval , SAA_O_time , saa_o_uvw , g_SDLD_O = original_sample_average_approximation( 1 , param , earthquake_info=[magnitude,depth] )
# print(f'SDA : {SAA_O_objval}')
# print(f'SDA time : {SAA_O_time}')

# SAA_objval_list , SAA_o_list = solving_subproblem(fixed_u=saa_uvw[0], fixed_v=saa_uvw[1], fixed_w=saa_uvw[2], set_NO=1, params=param, earthquake_info=[magnitude,depth])
# SAA_O_objval_list , SAAO_o_list = solving_subproblem(fixed_u=saa_o_uvw[0], fixed_v=saa_o_uvw[1], fixed_w=saa_o_uvw[2], set_NO=1, params=param, earthquake_info=[magnitude,depth])

# with open(file_name, 'a') as f:
#     f.write(f'SAA : {SAA_objval}\n')
#     f.write(f'SAA time : {SAA_time}\n')
#     f.write(f'SAA_original : {SAA_O_objval}\n')
#     f.write(f'SAA_original : {SAA_O_time}\n')
#     f.write(f"-"*50+'\n')
#     f.write(f"-"*50+'\n')
#     f.write(f'After solving the original formulation, and the formulation incident with uncertain weather condition, we have the value of first-stage decision variables\n')
#     f.write(f'We fix the value of first-stage decision variables, and solve the subproblem of different scenarios\n')

# for s in range(12):
#     with open(file_name, 'a') as f:
#         f.write(f'Scenario {s+1} : \n')
#         f.write(f'SAA : {SAA_objval_list[s]}, unmet demand : {np.mean(SAA_o_list[s])}\n')
#         f.write(f'SAA_original : {SAA_O_objval_list[s]}, unmet demand : {np.mean(SAAO_o_list[s])}\n')

# with open(file_name, 'a') as f:
#     f.write(f"-"*50+'\n')
#     f.write(f"-"*50+'\n')
#     f.write(f'On average, the objective function value and the unmet demand are shown as follows:\n')
#     f.write(f'SAA : {np.mean(SAA_objval_list)}, unmet demand : {np.mean(SAA_o_list)}\n')
#     f.write(f'SAA_original : {np.mean(SAA_O_objval_list)}, unmet demand : {np.mean(SAAO_o_list)}\n')

set_NO = 1

demand_list , GP_location_list = read_csv_files(set_NO,"GP", S)
DP_location_list = read_csv_files(set_NO,"DP", S)
LP_location_list = read_csv_files(set_NO,"LP", S)
weather_list = read_csv_files(set_NO,"Wind", S)
EQ_location_list = read_csv_files(set_NO,"EQ", S)

smalldrones = SmallDrone("SD")
largedrones = LargeDrone("LD")

Q_SD = smalldrones.max_payload
Q_LD = largedrones.max_payload

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ SDA ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
GP_location = GP_location_list[0]
DP_location = DP_location_list[0]
LP_location = LP_location_list[0]
GP_DP_matrix = construct_distance_angle_matrix(GP_location, DP_location)
GP_LP_matrix = construct_distance_angle_matrix(GP_location, LP_location)
DP_LP_matrix = construct_distance_angle_matrix(DP_location, LP_location)

magnitude_list = [7.5, 6.8, 7.7, 7.6, 7.7, 6.8, 7.3, 7.0, 6.8, 7.7, 6.8, 7.7]
depth_list = [5.3, 14.4, 5.4, 16, 12.7, 8.7, 12.6, 8.2, 12.7, 12.7, 16.9, 8.7]

lbs = []
ovals = []
for s in range(S):
    q = demand_list[s]
    EQ_location = EQ_location_list[s]
    mag = magnitude_list[s]
    depth = depth_list[s]

    EQ_LP_matrix = construct_distance_angle_matrix(LP_location, EQ_location)
    EQ_DP_matrix = construct_distance_angle_matrix(DP_location, EQ_location)

    DP_affected_percentage_array = np.zeros((num_candidates_depots,1))
    for d in range(num_candidates_depots):
        distance , angle = EQ_DP_matrix[d][0]
        DP_affected_percentage_array[d] = affected_percentage(mag, depth , distance)
    
    LP_affected_percentage_array = np.zeros((num_candidates_launch_points,1))
    for l in range(num_candidates_launch_points):
        distance , angle = EQ_LP_matrix[l][0]
        LP_affected_percentage_array[l] = affected_percentage(mag, depth , distance)

    DP_LP_matrix_new = np.zeros((num_candidates_depots,num_candidates_launch_points))
    for d in range(num_candidates_depots):
        for l in range(num_candidates_launch_points):
            DP_LP_matrix_new[d,l] = DP_LP_matrix[d,l][0] * ( 1 + DP_affected_percentage_array[d] + LP_affected_percentage_array[l] )
    
    wind_array = weather_list[s][0]
    wind_speed = wind_array[0]
    wind_angle = wind_array[1]

    # Create a new model for scenario solving
    m1 = gp.Model("scenario_solving")
    m1.setParam('OutputFlag', 0)

    # Add first stage variables
    u = m1.addVars(num_candidates_depots, vtype=gp.GRB.BINARY, name="depot")
    v = m1.addVars(num_candidates_launch_points, vtype=gp.GRB.BINARY, name="launch_point")
    w = m1.addVars(num_candidates_depots, num_candidates_launch_points, vtype=gp.GRB.BINARY, name="LP_to_DP")

    # v.BranchPriority = 1
    # u.BranchPriority = 3
    # w.BranchPriority = 1

    # Add second stage variables
    g_SD = m1.addVars(K_SD, vtype=gp.GRB.BINARY, name="small_drone_usage")
    g_LD = m1.addVars(K_LD, vtype=gp.GRB.BINARY, name="large_drone_usage")
    x = m1.addVars(num_gathering_points, num_candidates_launch_points + num_candidates_depots, K_SD, vtype=gp.GRB.BINARY, name="small_drone_assignment")
    y = m1.addVars(num_gathering_points, num_candidates_depots, K_LD, vtype=gp.GRB.BINARY, name="large_drone_assignment")
    o = m1.addVars(num_gathering_points, lb=0, vtype=gp.GRB.CONTINUOUS, name="unmet_demand")

    # Add objective function
    m1.setObjective(gp.quicksum(pi * o[i] for i in range(num_gathering_points)) +
                    gp.quicksum(c_SD * g_SD[k] for k in range(K_SD)) +
                    gp.quicksum(c_LD * g_LD[k] for k in range(K_LD)),
                    gp.GRB.MINIMIZE)
    # Add constraints
    m1.addConstr(gp.quicksum(u[d] for d in range(num_candidates_depots)) <= e)
    m1.addConstr(gp.quicksum(v[l] for l in range(num_candidates_launch_points)) <= p)
    for l in range(num_candidates_launch_points):
        m1.addConstr(gp.quicksum(w[d,l] for d in range(num_candidates_depots)) == v[l])
        m1.addConstr(gp.quicksum( q[i][0] * x[i,l,k] for i in range(num_gathering_points) for k in range(K_SD)) <= Q_T * v[l] )

    for k in range(K_SD):
        m1.addConstr(gp.quicksum(x[i,l,k] for i in range(num_gathering_points) for l in range(num_candidates_launch_points + num_candidates_depots)) <= g_SD[k])
        if k < K_SD - 1:
            m1.addConstr(g_SD[k] >= g_SD[k+1]) # Remove symmetry
        for l in range(num_candidates_launch_points):
            total_airtime = 0
            for i in range(num_gathering_points):
                distance , angle = GP_LP_matrix[i,l] # retrieve the distance and angle between the current launch point and the gathering point
                air_speed_SD = airspeed_calculation( d = distance , m_pack= Q_SD, drone=smalldrones) # calculates the maximum airspeed (km/hr) of the small drone
                if air_speed_SD <= wind_speed:
                    m1.addConstr(x[i,l,k] == 0)
                    total_airtime += 0
                else:
                    total_airtime += x[i,l,k] * (calculate_time_to_deliver( air_speed_SD , angle , wind_speed, wind_angle , distance ) + tau)
            m1.addConstr( gp.quicksum( w[d,l] * DP_LP_matrix_new[d,l]/v_T for d in range(num_candidates_depots)) + total_airtime <= T )
        for l in range(num_candidates_depots):
            total_airtime = 0
            for i in range(num_gathering_points):
                distance , angle = GP_DP_matrix[i,l] # retrieve the distance and angle between the current launch point and the gathering point
                air_speed_SD = airspeed_calculation( d = distance , m_pack= Q_SD, drone=smalldrones) # calculates the maximum airspeed (km/hr) of the small drone
                if air_speed_SD <= wind_speed:
                    m1.addConstr(x[i,l+num_candidates_launch_points,k] == 0)
                    total_airtime += 0
                else:
                    total_airtime += x[i,l+num_candidates_launch_points,k] * (calculate_time_to_deliver( air_speed_SD , angle , wind_speed, wind_angle , distance ) + tau)
            m1.addConstr( total_airtime <= T )

    for k in range(K_LD):
        m1.addConstr(gp.quicksum(y[i,d,k] for i in range(num_gathering_points) for d in range(num_candidates_depots)) <= g_LD[k])
        if k < K_LD - 1:
            m1.addConstr(g_LD[k] >= g_LD[k+1]) # Remove symmetry
        for d in range(num_candidates_depots):
            total_airtime = 0
            for i in range(num_gathering_points):
                distance , angle = GP_DP_matrix[i,d] # retrieve the distance and angle between the current launch point and the gathering point
                air_speed_LD = airspeed_calculation( d = distance , m_pack= Q_LD, drone=largedrones) # calculates the maximum airspeed (km/hr) of the small drone
                if air_speed_LD <= wind_speed:
                    m1.addConstr(y[i,d,k] == 0)
                    total_airtime += 0
                else:
                    total_airtime += y[i,d,k] * calculate_time_to_deliver( air_speed_LD , angle , wind_speed, wind_angle , distance )
            m1.addConstr( total_airtime <= T )

    for i in range(num_gathering_points):
        m1.addConstr( q[i][0] 
                    -  Q_SD * gp.quicksum( x[i,l,k] for l in range(num_candidates_depots+num_candidates_launch_points) for k in range(K_SD)) 
                    -  Q_LD * gp.quicksum( y[i,d,k] for d in range(num_candidates_depots) for k in range(K_LD))
                    <= o[i])
        for d in range(num_candidates_depots):
            for k in range(K_LD):
                m1.addConstr(y[i,d,k] <= u[d])
            for k in range( K_SD ):
                m1.addConstr(x[i,(num_candidates_launch_points+d),k] <= u[d])
        for l in range(num_candidates_launch_points):
            for k in range(K_SD):
                m1.addConstr(x[i,l,k] <= v[l])
 # Optimize the model
    m1.optimize()
    # Retrieve and store optimal variable values
    if m1.status == gp.GRB.OPTIMAL:
        u_values = [u[d].X for d in range(num_candidates_depots)]
        v_values = [v[l].X for l in range(num_candidates_launch_points)]
        w_values = np.zeros((num_candidates_depots,num_candidates_launch_points))
        o_values = np.zeros(num_gathering_points)
        for d in range(num_candidates_depots):
            for l in range(num_candidates_launch_points):
                w_values[d,l] = w[d,l].X
        for i in range(num_gathering_points):
            o_values[i] = o[i].X
    
        lbs.append(m1.objVal)
        ovals.append(np.mean(o_values))

    with open(file_name, 'a') as f:
        f.write(f'Scenario {s+1} : \n')
        f.write(f'EXPI : {m1.objVal}, unmet demand : {np.mean(o_values)}\n')

LB = np.mean(lbs)

with open(file_name, 'a') as f:
    f.write(f"-"*50+'\n')
    f.write(f"-"*50+'\n')
    f.write(f'On average, the objective function value and the unmet demand are shown as follows:\n')
    f.write(f'EVPI: {np.mean(LB)}, unmet demand : {np.mean(ovals)}\n')

Test 1 : max_LP = 10 , max_DP = 4 , num_SmallDrone = 30 , num_LargeDrone = 5


C:\Users\Yuyang\AppData\Local\Temp\ipykernel_25248\3282059491.py:134: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  DP_LP_matrix_new[d,l] = DP_LP_matrix[d,l][0] * ( 1 + DP_affected_percentage_array[d] + LP_affected_percentage_array[l] )


AttributeError: Unable to retrieve attribute 'objVal'

In [ ]:
# # In this part, I want to do the equity test, to show the significance of considering the equity.
# # I divide the test into 3 parts, the first is test the equity by adding a constraint, the second is to test the equity by adding a penalty term, the third is to test the equity by adding a constraint and a penalty term.

# from SAA_Equity import *

# directory = './output/'
# if not os.path.exists(directory):
#     os.makedirs(directory)

# test_no = 5

# magnitude = [7.5, 6.8, 7.7, 7.6, 7.7, 6.8, 7.3, 7.0, 6.8, 7.7, 6.8, 7.7]
# depth = [5.3, 14.4, 5.4, 16, 12.7, 8.7, 12.6, 8.2, 12.7, 12.7, 16.9, 8.7]

# file_name = f'{directory}output_Equity_test_{test_no}.out'

# param = Parameters()
# param.printlevel = 0
# param.num_candidates_launch_points = 50
# param.num_candidates_depots = 30
# param.e = 5
# param.p = 20
# param.num_gathering_points = 30

# max_LP = param.p
# max_DP = param.e
# num_SmallDrone = param.K_SD
# num_LargeDrone = param.K_LD


# print(f'Test {test_no} : num_GP = {param.num_gathering_points} , max_LP = {max_LP} , max_DP = {max_DP} , num_SmallDrone = {num_SmallDrone} , num_LargeDrone = {num_LargeDrone} , Time Frame = {param.T} hrs')


# # Starting the test
# Equity_Constr_objval , Equity_Constr_time , _ , _ , Equity_Constr_O = SAA_equity_constr( test_no , param , earthquake_info=[magnitude,depth])
# print(f'Equity Constraint : {Equity_Constr_objval}')
# print(f'Equity Constraint Time : {Equity_Constr_time}')

# Equity_Obj_objval , Equity_Obj_time , _ , _ , Equity_Obj_O = SAA_equity_obj( test_no , param , earthquake_info=[magnitude,depth])
# print(f'Equity Objective : {Equity_Obj_objval}')
# print(f'Equity Objective Time : {Equity_Obj_time}')

# Equity_Comb_objval , Equity_Comb_time , _ , _ , Equity_Comb_O = SAA_equity_obj_constr( test_no , param , earthquake_info=[magnitude,depth])
# print(f'Equity Combined : {Equity_Comb_objval}')
# print(f'Equity Combined Time : {Equity_Comb_time}')

# SAA_objval , SAA_time , saa_uvw , g_SDLD , SAA_O = sample_average_approximation( test_no , param , earthquake_info=[magnitude,depth])
# print(f'SAA : {SAA_objval}')
# print(f'SAA time : {SAA_time}')

# with open(file_name, 'a') as f:
#     f.write(f'Test {test_no} : num_GP = {param.num_gathering_points} , max_LP = {max_LP} , max_DP = {max_DP} , num_SmallDrone = {num_SmallDrone} , num_LargeDrone = {num_LargeDrone} , Time Frame = {param.T} hrs\n')
#     f.write(f'Equity Constraint : {Equity_Constr_objval}\n')
#     f.write(f'Equity Constraint Time : {Equity_Constr_time}\n')
#     f.write(f'Equity Objective : {Equity_Obj_objval}\n')
#     f.write(f'Equity Objective Time : {Equity_Obj_time}\n')
#     f.write(f'Equity Combined : {Equity_Comb_objval}\n')
#     f.write(f'Equity Combined Time : {Equity_Comb_time}\n')
#     f.write(f'SAA : {SAA_objval}\n')
#     f.write(f'SAA time : {SAA_time}\n')
#     f.write(f"-"*50+'\n')
#     f.write(f"-"*50+'\n')
#     f.write(f'After solving the 3 formulations, we have the unmet demand for each scenarios\n')
#     f.write(f'The average unmet demand, and difference of demands of each scenario are shown as follows\n')

# for s in range(12):
#     with open(file_name, 'a') as f:
#         f.write(f'Scenario {s+1} : \n')
#         f.write(f'Equity Constraint : {np.mean(Equity_Constr_O[s])}, Maximal Difference : {np.max(Equity_Constr_O[s])-np.min(Equity_Constr_O[s])}\n')
#         f.write(f'Equity Objective : {np.mean(Equity_Obj_O[s])}, Maximal Difference : {np.max(Equity_Obj_O[s])-np.min(Equity_Obj_O[s])}\n')
#         f.write(f'Equity Combined : {np.mean(Equity_Comb_O[s])}, Maximal Difference : {np.max(Equity_Comb_O[s])-np.min(Equity_Comb_O[s])}\n')
#         f.write(f'SAA : {np.mean(SAA_O[s])}, Maximal Difference : {np.max(SAA_O[s])-np.min(SAA_O[s])}\n')

# with open(file_name, 'a') as f:
#     f.write(f"-"*50+'\n')
#     f.write(f"-"*50+'\n')

In [ ]:
# max_LP_list = [4 , 5 , 6]
# max_DP_list = [1 , 2]
# num_SmallDrone_list = [10 , 15 , 20]
# num_LargeDrone_list = [1 , 2 , 3]

# magnitude = [7.5, 6.8, 7.7, 7.6, 7.7, 6.8, 7.3, 7.0, 6.8, 7.7, 6.8, 7.7]
# depth = [5.3, 14.4, 5.4, 16, 12.7, 8.7, 12.6, 8.2, 12.7, 12.7, 16.9, 8.7]

# total_test = 3

# directory = './output/'
# if not os.path.exists(directory):
#     os.makedirs(directory)

# for test_NO in range(total_test):
#     set_NO = test_NO + 1
#     # Create a file name that includes the test number
#     file_name = f'{directory}output_test_{set_NO}.out'
#     for max_LP in max_LP_list:
#         for max_DP in max_DP_list:
#             for n in range(len(num_SmallDrone_list)):

#                 num_SmallDrone = num_SmallDrone_list[n]
#                 num_LargeDrone = num_LargeDrone_list[n]

#                 param = Parameters()
#                 param.printlevel = 0
#                 print(f'Test {set_NO} : max_LP = {max_LP} , max_DP = {max_DP} , num_SmallDrone = {num_SmallDrone} , num_LargeDrone = {num_LargeDrone}')
#                 with open(file_name, 'a') as f:
#                     f.write(f'Test {set_NO} : max_LP = {max_LP} , max_DP = {max_DP} , num_SmallDrone = {num_SmallDrone} , num_LargeDrone = {num_LargeDrone}\n')

#                 param.K_SD = num_SmallDrone
#                 param.K_LD = num_LargeDrone
#                 param.e = max_DP
#                 param.p = max_LP

#                 SAA_objval , SAA_time , saa_uvw , g_SDLD = sample_average_approximation(set_NO , param , earthquake_info=[magnitude,depth])

#                 print(f'SAA : {SAA_objval}')
#                 print(f'SAA time : {SAA_time}')
#                 with open(file_name, 'a') as f:
#                     f.write(f'SAA : {SAA_objval}\n')
#                     f.write(f'SAA time : {SAA_time}\n')

#                 SDA_objval , SDA_time , sda_uvw= scenario_decomposition_algorithm(set_NO , param, earthquake_info=[magnitude,depth])
#                 with open(file_name, 'a') as f:
#                     f.write(f'SDA : {SDA_objval}\n')
#                     f.write(f'SDA time : {SDA_time}\n')
                
#                 print(f'SDA : {SDA_objval}')
#                 print(f'SDA time : {SDA_time}')

#                 # Bender_objval , Bender_time , bender_uvw= benders_decomposition(set_NO , param, earthquake_info=[magnitude,depth])
#                 # print(f'Benders : {Bender_objval}')
#                 # print(f'Benders Time : {Bender_time}')

#                 # with open(file_name, 'a') as f:
#                 #     f.write(f'Benders : {Bender_objval}\n')
#                 #     f.write(f'Benders Time : {Bender_time}\n')
#                 print('----------------------------------------')
#                 print('----------------------------------------')
#                 with open(file_name, 'a') as f:
#                     f.write('----------------------------------------\n')

max_LP_list = [4 , 5 , 6]
max_DP_list = [1 , 2]
num_SmallDrone_list = [10 , 15 , 20]
num_LargeDrone_list = [1 , 2 , 3]

magnitude = [7.5, 6.8, 7.7, 7.6, 7.7, 6.8, 7.3, 7.0, 6.8, 7.7, 6.8, 7.7]
depth = [5.3, 14.4, 5.4, 16, 12.7, 8.7, 12.6, 8.2, 12.7, 12.7, 16.9, 8.7]

for test_NO in range(1):
    set_NO = test_NO + 1
    # Create a file name that includes the test number
    # file_name = f'{directory}output_test_{set_NO}.out'
    for max_LP in max_LP_list:
        for max_DP in max_DP_list:
            for n in range(len(num_SmallDrone_list)):

                num_SmallDrone = num_SmallDrone_list[n]
                num_LargeDrone = num_LargeDrone_list[n]

                param = Parameters()
                param.printlevel = 0
                print(f'Test {set_NO} : max_LP = {max_LP} , max_DP = {max_DP} , num_SmallDrone = {num_SmallDrone} , num_LargeDrone = {num_LargeDrone}')

                param.K_SD = num_SmallDrone
                param.K_LD = num_LargeDrone
                param.e = max_DP
                param.p = max_LP

                # SAA_objval , SAA_time , saa_uvw , g_SDLD = sample_average_approximation(set_NO , param , earthquake_info=[magnitude,depth])

                # print(f'SAA : {SAA_objval}')
                # print(f'SAA time : {SAA_time}')

                Bender_objval , Bender_time , bender_uvw= benders_decomposition(set_NO , param, earthquake_info=[magnitude,depth])
                print(f'Benders : {Bender_objval}')
                print(f'Benders Time: {Bender_time}')


In [ ]:
# # Heuristic Algorithm
# from sklearn.cluster import KMeans

# # First step, we read the data from the data store
# max_LP_list = [4 , 5 , 6]
# max_DP_list = [1 , 2]
# num_SmallDrone_list = [10 , 15 , 20]
# num_LargeDrone_list = [1 , 2 , 3]

# num_S_list = [20 , 50 , 100]

# magnitude = [7.5, 6.8, 7.7, 7.6, 7.7, 6.8, 7.3, 7.0, 6.8, 7.7, 6.8, 7.7]
# depth = [5.3, 14.4, 5.4, 16, 12.7, 8.7, 12.6, 8.2, 12.7, 12.7, 16.9, 8.7]

# total_test = 3

# directory = './output/'
# if not os.path.exists(directory):
#     os.makedirs(directory)

# for test_NO in range(total_test):
#     set_NO = test_NO + 1
#     # Create a file name that includes the test number
#     file_name = f'{directory}Heuristic_output_test_{set_NO}.out'
#     for max_LP in max_LP_list:
#         for max_DP in max_DP_list:
#             for n in range(len(num_SmallDrone_list)):

#                 # Create a file name that includes the test number
#                 num_SmallDrone = num_SmallDrone_list[n]
#                 num_LargeDrone = num_LargeDrone_list[n]
                
#                 params = Parameters()
#                 params.printlevel = 0
#                 params.K_SD = num_SmallDrone
#                 params.K_LD = num_LargeDrone

#                 params.p = max_LP
#                 params.e = max_DP
                
#                 S = params.n_scenarios
#                 pi = params.pi
#                 T = params.T
#                 e = params.e
#                 p = params.p
#                 Q_T = params.Q_T
#                 c_SD = params.c_SD
#                 c_LD = params.c_LD
#                 v_T = params.v_T
#                 tau = params.tau
#                 K_SD = params.K_SD
#                 K_LD = params.K_LD
#                 num_candidates_depots = params.num_candidates_depots
#                 num_candidates_launch_points = params.num_candidates_launch_points
#                 num_gathering_points = params.num_gathering_points
#                 printlevel = params.printlevel

#                 gp_demand_and_location, D_location, LP_location , earthquake_location = generate_all_locations( S=S , num_GP=num_gathering_points , cand_D=num_candidates_depots , cand_LP=num_candidates_launch_points)
#                 gp_demand = gp_demand_and_location[0]
#                 gp_location = gp_demand_and_location[1]

#                 for s in range(S):
#                         save_to_csv(set_NO ,"GP", s + 1, gp_demand, gp_location)
#                         save_to_csv(set_NO ,"DP", s + 1, None, D_location)
#                         save_to_csv(set_NO ,"LP", s + 1, None, LP_location)
#                         wind_speed , wind_angle = generate_wind_speed_and_angle()
#                         wind_array = np.array([[wind_speed],[wind_angle]])
#                         save_to_csv(set_NO ,"Wind", s + 1, None, None, wind_array)
#                         save_to_csv(set_NO ,"EQ", s + 1, demand_array=None, location_array=earthquake_location[s])

#                 demand_list , GP_location_list = read_csv_files(set_NO,"GP", S)
#                 DP_location_list = read_csv_files(set_NO,"DP", S)
#                 LP_location_list = read_csv_files(set_NO,"LP", S)
#                 weather_list = read_csv_files(set_NO,"Wind", S)
#                 EQ_location_list = read_csv_files(set_NO,"EQ", S)

#                 # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ GP Cluster ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                 time_now = time.time()
#                 kmeans = KMeans(n_clusters=p).fit(GP_location_list[0])

#                 # Get the cluster labels
#                 labels = kmeans.labels_

#                 # Initialize a list to hold the clusters
#                 clusters = [[] for _ in range(p)]

#                 # Assign points to the corresponding cluster
#                 for point, label in zip(GP_location_list[0], labels):
#                     clusters[label].append(point.tolist())
#                 # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Assign one LP to each Cluster ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                 # Assign each candidate launch point to the closest cluster
#                 LP_Clst = []
#                 for lp in LP_location_list[0]:
#                     dist_list = []
#                     for i, cluster in enumerate(clusters):
#                         clst_dist = 0
#                         for gp in cluster:
#                             clst_dist += np.linalg.norm(np.array(gp) - np.array(lp))
#                         dist_list.append(np.mean(clst_dist))
#                     LP_Clst.append([dist_list.index(min(dist_list)),min(dist_list)])

#                 # Find the closest launch point to open
#                 # Initialize a dictionary to store the smallest second index for each first index
#                 min_values = {}

#                 # Iterate through the list with enumeration to get the index of each point
#                 for idx, point in enumerate(LP_Clst):
#                     index, value = point
#                     # Check if the index is already in the dictionary
#                     if index not in min_values:
#                         min_values[index] = (point, idx)
#                     else:
#                         # Update if the current value is smaller
#                         if value < min_values[index][0][1]:
#                             min_values[index] = (point, idx)

#                 # Convert the dictionary values back to a list with the original index included
#                 LP_open = [(idx, point) for point, idx in min_values.values()]
#                 LP_open_locs = [LP_location_list[0][idx] for idx, _ in LP_open]
#                 v = np.zeros(num_candidates_launch_points)
#                 for idx, _ in LP_open:
#                     v[idx] = 1

#                 # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ LP Cluster ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                 kmeans_LP = KMeans(n_clusters=e).fit(LP_open_locs)

#                 # Get the cluster labels
#                 labels_LP = kmeans_LP.labels_

#                 # Initialize a list to hold the clusters
#                 clusters_LP = [[] for _ in range(e)]

#                 # Assign points to the corresponding cluster
#                 for point, label in zip(LP_open_locs, labels_LP):
#                     clusters_LP[label].append(point.tolist())

#                 # Open the depot closest to the open launch point and gathering points
#                 DP_clst = []

#                 DP_total_dist = []
#                 for dp in DP_location_list[0]:
#                     total_dist = 0
#                     DP_dist_list = []
#                     for label, cluster in enumerate(clusters_LP):
#                         clst_LP_dist = 0
#                         clst_GP_dist = 0
#                         for lp in cluster:
#                             clst_LP_dist += np.linalg.norm(np.array(lp) - np.array(dp))
#                             # Find the index of current LP in the original LP list
#                             LP_idx = np.where(np.all(LP_location_list[0] == lp,axis=1))[0][0]
#                             # Find which cluster of GP the current LP belongs to
#                             GP_clst_idx = LP_Clst[LP_idx][0]
#                             # Find the GPs in the cluster
#                             GP_clst = clusters[GP_clst_idx]
#                             # Calculate the distance between the current DP and the GPs in the cluster
#                             for gp in GP_clst:
#                                 clst_GP_dist += np.linalg.norm(np.array(gp) - np.array(dp))
#                         DP_dist_list.append(clst_LP_dist + clst_GP_dist)
#                     DP_clst.append([DP_dist_list.index(min(DP_dist_list)),min(DP_dist_list)])

#                 # Find the closest depot to open
#                 # Initialize a dictionary to store the smallest second index for each first index
#                 min_values = {}

#                 # Iterate through the list with enumeration to get the index of each point
#                 for idx, point in enumerate(DP_clst):
#                     index, value = point
#                     # Check if the index is already in the dictionary
#                     if index not in min_values:
#                         min_values[index] = (point, idx)
#                     else:
#                         # Update if the current value is smaller
#                         if value < min_values[index][0][1]:
#                             min_values[index] = (point, idx)
                            
#                 DP_open = [(idx, point) for point, idx in min_values.values()]
#                 w = np.zeros((num_candidates_depots,num_candidates_launch_points))
#                 u = np.zeros(num_candidates_depots)
#                 for idx , lb_dist in DP_open:
#                     u[idx] = 1
#                     for lp in clusters_LP[lb_dist[0]]:
#                         w[idx][np.where(np.all(LP_location_list[0] == lp,axis=1))[0][0]] = 1

#                 # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Solve the subproblem ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                 H_objval_list , H_o_list = solving_subproblem(fixed_u=list(u), fixed_v=list(v), fixed_w=w, set_NO=set_NO, params=params, earthquake_info=[magnitude,depth])
#                 H_time = time.time() - time_now
#                 print(f'Test {set_NO} : num_GP = {params.num_gathering_points} , max_LP = {max_LP} , max_DP = {max_DP} , num_SmallDrone = {num_SmallDrone} , num_LargeDrone = {num_LargeDrone} , Time Frame = {params.T} hrs\n')
#                 print(f'Heurstic : {np.mean(H_objval_list)}') 
#                 print(f'Heuristic time : {H_time}')

#                 # SAA_objval , SAA_time , saa_uvw , g_SDLD , o_value = sample_average_approximation( set_NO , params , earthquake_info=[magnitude,depth])
#                 # print(f'SAA : {SAA_objval}')
#                 # print(f'SAA time : {SAA_time}')

#                 # SDA_objval , SDA_time , sda_uvw= scenario_decomposition_algorithm(set_NO , params, earthquake_info=[magnitude,depth])
#                 # print(f'SDA : {SDA_objval}')
#                 # print(f'SDA time : {SDA_time}')
                
#                 # Recording results to file
#                 with open(file_name, 'a') as f:
#                     f.write(f'Test {set_NO} : num_GP = {params.num_gathering_points} , max_LP = {max_LP} , max_DP = {max_DP} , num_SmallDrone = {num_SmallDrone} , num_LargeDrone = {num_LargeDrone} , Time Frame = {params.T} hrs\n')
#                     f.write(f'Heuristic : {np.mean(H_objval_list)}\n')
#                     f.write(f'Heuristic time : {H_time}\n')
#                     f.write("-" * 50 + '\n')
#                     f.write("-" * 50 + '\n')